In [1]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection && pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4 MB 1.2 kB/s 
     |████████████████████████████████| 6.6 MB 18.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.1+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.1+cu101 which is incompatible.
     |████████████████████████████████| 390 kB 9.8 MB/s 
     |█████████████████

In [1]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import matplotlib.pyplot as plt
import cv2
import numpy as np

%matplotlib inline

1.5.1+cu101 True
2.18.1
11.1
GCC 7.5


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download original weights

In [3]:
! mkdir -p /mmdetection/checkpoints/
! wget -nc https://download.openmmlab.com/mmdetection/v2.0/vfnet/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-b5f6da5e.pth \
       -O /mmdetection/checkpoints/vfnet.pth

File ‘/mmdetection/checkpoints/vfnet.pth’ already there; not retrieving.


In [10]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Choose to use a config and initialize the detector
config = '/content/drive/MyDrive/rosneft_hack/deers_exps/configs/vfnet_aug_v2.py'
# Setup a checkpoint file to load
checkpoint = '/content/drive/MyDrive/rosneft_hack/deers_exps/epoch_48.pth'

In [5]:
# initialize the detector
# model = init_detector(config, checkpoint, device='cuda')

## Change the config

In [11]:
albu_train_transforms = [
    dict(
        type = 'RandomCrop',
         height = 800,
         width = 1333,
         p = 0.5,
    ),
    dict(
        type = 'ColorJitter',
         brightness = 0.3,
         contrast = 0.3,
         saturation = 0.3,
         hue = 0.3,
         p = 0.5,
    )
]

In [12]:
from mmcv import Config
cfg = Config.fromfile(config)

from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'CocoDataset'
cfg.classes = ('fawn', 'reindeer')
cfg.data_root = '/content/drive/MyDrive/rosneft_hack/data'
#cfg.train_pipeline.insert(
#    4,
#    dict(
#        type='CutOut',
#        n_holes=(0, 4),
#        cutout_ratio=[(0.01, 0.008), (0.005, 0.01), (0.005, 0.01), (0.01, 0.01)],
#    )
#)
cfg.train_pipeline.insert(
   4,
   dict(
       type = 'RandomAffine',
        max_rotate_degree = 30.0,
        max_translate_ratio = 0.2,
        scaling_ratio_range = (0.5, 1.5)

   )
)
cfg.train_pipeline.insert(
   5,
   dict(
       type = 'RandomFlip',
        flip_ratio = 0.5,
        direction = 'horizontal',
   )
)

cfg.train_pipeline.insert(
    6,
    dict(
        type= 'Albu',
        transforms= albu_train_transforms,
        bbox_params=dict(
                type='BboxParams',
                format='coco',
                label_fields=['gt_labels'],
                min_visibility=0.0,
                filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True)
)

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 4

cfg.data.test.type = 'CocoDataset'
cfg.data.test.data_root = cfg.data_root
cfg.data.test.ann_file = 'test/coco_annotations.json'
cfg.data.test.img_prefix = 'test/data/'
cfg.data.test.classes = cfg.classes

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = cfg.data_root
cfg.data.train.ann_file = 'train/coco_annotations.json'
cfg.data.train.img_prefix = 'train/data/'
cfg.data.train.classes = cfg.classes

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = cfg.data_root
cfg.data.val.ann_file = 'all_data/coco_annotations.json'
cfg.data.val.img_prefix = 'all_data/images/'
cfg.data.val.classes = cfg.classes


cfg.model.bbox_head.num_classes = len(cfg.classes)

# We can still use the pre-trained model
cfg.load_from = checkpoint

# Set up working dir to save files and logs.
cfg.work_dir = './deers_exps/'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 1e-4
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = ['bbox', ]
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0, ]


# We can initialize the logger for training and have a look
# at the final config used for training
#print(f'Config:\n{cfg.pretty_text}')

In [16]:
cfg.dump('/content/drive/MyDrive/rosneft_hack/deers_exps/configs/vfnet_aug_v2.py')

In [7]:
checkpoint = '/content/drive/MyDrive/rosneft_hack/deers_exps/epoch_48.pth'
config = '/content/drive/MyDrive/rosneft_hack/deers_exps/configs/vfnet_aug_v2.py'

model = init_detector(config, checkpoint=checkpoint, device='cuda')

load checkpoint from local path: /content/drive/MyDrive/rosneft_hack/deers_exps/epoch_48.pth


2021-11-20 20:52:59,213 - root - INFO - ModulatedDeformConvPack backbone.layer2.0.conv2 is upgraded to version 2.
2021-11-20 20:52:59,220 - root - INFO - ModulatedDeformConvPack backbone.layer2.1.conv2 is upgraded to version 2.
2021-11-20 20:52:59,225 - root - INFO - ModulatedDeformConvPack backbone.layer2.2.conv2 is upgraded to version 2.
2021-11-20 20:52:59,230 - root - INFO - ModulatedDeformConvPack backbone.layer2.3.conv2 is upgraded to version 2.
2021-11-20 20:52:59,235 - root - INFO - ModulatedDeformConvPack backbone.layer3.0.conv2 is upgraded to version 2.
2021-11-20 20:52:59,242 - root - INFO - ModulatedDeformConvPack backbone.layer3.1.conv2 is upgraded to version 2.
2021-11-20 20:52:59,248 - root - INFO - ModulatedDeformConvPack backbone.layer3.2.conv2 is upgraded to version 2.
2021-11-20 20:52:59,253 - root - INFO - ModulatedDeformConvPack backbone.layer3.3.conv2 is upgraded to version 2.
2021-11-20 20:52:59,259 - root - INFO - ModulatedDeformConvPack backbone.layer3.4.conv2 

In [13]:
!nvidia-smi

Sat Nov 20 20:55:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    35W / 250W |  14351MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Let's train

In [14]:
import mmcv
import os.path as osp
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model,
    train_cfg=cfg.get('train_cfg'),
    test_cfg=cfg.get('test_cfg'),
)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
print(datasets[0].CLASSES)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.13s)
creating index...
index created!
('fawn', 'reindeer')
loading annotations into memory...


2021-11-20 20:55:12,643 - mmdet - INFO - load checkpoint from local path: /content/drive/MyDrive/rosneft_hack/deers_exps/epoch_48.pth


Done (t=0.30s)
creating index...
index created!


2021-11-20 20:55:13,838 - root - INFO - ModulatedDeformConvPack backbone.layer2.0.conv2 is upgraded to version 2.
2021-11-20 20:55:13,845 - root - INFO - ModulatedDeformConvPack backbone.layer2.1.conv2 is upgraded to version 2.
2021-11-20 20:55:13,850 - root - INFO - ModulatedDeformConvPack backbone.layer2.2.conv2 is upgraded to version 2.
2021-11-20 20:55:13,856 - root - INFO - ModulatedDeformConvPack backbone.layer2.3.conv2 is upgraded to version 2.
2021-11-20 20:55:13,862 - root - INFO - ModulatedDeformConvPack backbone.layer3.0.conv2 is upgraded to version 2.
2021-11-20 20:55:13,869 - root - INFO - ModulatedDeformConvPack backbone.layer3.1.conv2 is upgraded to version 2.
2021-11-20 20:55:13,876 - root - INFO - ModulatedDeformConvPack backbone.layer3.2.conv2 is upgraded to version 2.
2021-11-20 20:55:13,881 - root - INFO - ModulatedDeformConvPack backbone.layer3.3.conv2 is upgraded to version 2.
2021-11-20 20:55:13,887 - root - INFO - ModulatedDeformConvPack backbone.layer3.4.conv2 

[                                 ] 1/534, 0.5 task/s, elapsed: 2s, ETA:  1087s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 534/534, 3.3 task/s, elapsed: 164s, ETA:     0s

2021-11-21 03:23:11,239 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.23s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=61.95s).
Accumulating evaluation results...


2021-11-21 03:24:14,564 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.399
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.590
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.465
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.512
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.428
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.428
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.428
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.185
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.228
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.02s).


In [15]:
!cp -R /content/deers_exps/epoch_24.pth /content/drive/MyDrive/rosneft_hack/deers_exps/epoch_72.pth

In [16]:
!cp -R /content/deers_exps/latest.pth /content/drive/MyDrive/rosneft_hack/deers_exps/latest.pth